`env:talib`

# All Functions for indicator calculation and machine learning

In [2]:
# General Modules
import numpy as np
import pandas as pd
import sqlite3
import datetime

# Indicator Calculation modules
import talib
import math
from sklearn.metrics import r2_score
from pathlib import Path
pd.options.mode.chained_assignment = None  # default='warn'

# # Machine learning calculation modules
# import tensorflow as tf
# import re
# from keras.preprocessing.sequence import TimeseriesGenerator
# import json

# # Google colab modules
# from google.colab import drive
# import sys
# drive._mount('/content/gdrive', force_remount=True)
# ROOT_PATH = './gdrive/MyDrive/#PROJECT/idx/'
# sys.path.append(ROOT_PATH)

def stock_list_100_highestrank_and_availability():
    '''Return a list of idx stock that have
    highest availability and most popular.
    Algorithm:
    - Get 150 stock with highest data availability
    - From stock filtered above, get 100 most popular
        stock (by relative idx volume rank)
    '''
    tickers = ['BBRI', 'BUMI', 'ELTY', 'TLKM', 'BRPT', 'LPKR', 'BKSL', 'BMRI', 'KLBF', 'BTEK', 'ASRI', 'KIJA', 'FREN', 'ANTM', 'ASII', 'ADRO', 'MEDC', 'BHIT', 'PWON', 'PNLF', 'BNII', 'PTBA', 'TINS', 'ELSA', 'MLPL', 'DOID', 'KREN', 'CNKO', 'CTRA', 'META', 'ENRG', 'SMRA', 'MDLN', 'BBNI', 'APIC', 'BMTR', 'LSIP', 'MAPI', 'BSDE', 'INDF', 'BBKP', 'MNCN', 'CPIN', 'WIKA', 'BNBR', 'SSIA', 'BNGA', 'BEKS', 'ADHI', 'RAJA', 'DILD', 'BBCA', 'PBRX', 'DGIK', 'PNBN', 'INDY', 'ACES', 'MYOR', 'INCO', 'AKRA', 'TBLA', 'KPIG', 'GZCO', 'TOTL', 'UNVR', 'INKP', 'SMCB', 'MPPA', 'GJTL', 'INTA', 'JSMR', 'HMSP', 'CMNP', 'MASA', 'SRSN', 'BNLI', 'INAF', 'RALS', 'ADMG', 'UNTR', 'SCMA', 'ISAT', 'DSFI', 'BDMN', 'MTDL', 'SULI', 'TURI', 'SMGR', 'TMAS', 'MAIN', 'KAEF', 'EXCL', 'SMSM', 'LPPS', 'POLY', 'KBLI', 'UNSP', 'PKPK', 'BUDI', 'CSAP']
    return tickers

def excluded_stock():
    '''Stocks that causes error in the process.
    '''
    tickers = ['HDTX','TRIO','CMPP','NIPS','SUGI','TRIL','APIC']
    return tickers

def yfinance_db(stock_db='idx_raw.db', if_exists='replace', selected_stock_only=False, ROOT_PATH='./'):
    '''Fetch data from yfinance and store
    it into defined stock_db.
    '''
    stocks_db_conn = sqlite3.connect(f'{ROOT_PATH}{stock_db}')
    investing_metadata = stock_metadata(ROOT_PATH=ROOT_PATH)
    selected_stock = stock_list_100_highestrank_and_availability()
    
    count=0
    excluded_stock = []
    for index in range(len(investing_metadata)):
        tick = datetime.datetime.now()
        ticker = investing_metadata.loc[index]['ticker']
        
        if ticker not in selected_stock and selected_stock_only:
            continue    
            
        try:
            # Fetch and store data
            stock_object = yf.Ticker(f'{ticker}.JK')
            df = stock_object.history(period='max')
            df = df.reset_index()
            df['Date'] = df['Date'].apply(lambda x: x.value)
            df = df.drop(['Dividends', 'Stock Splits'], axis='columns')
            df = df.rename(columns={'Date':'time','Open':'open','High':'high','Low':'low','Close':'close'})
            df.to_sql(name=ticker, con=stocks_db_conn, index=False, if_exists=if_exists)
        except KeyError:
            excluded_stock.append(ticker)
        tock = datetime.datetime.now()
        print(f'{tock-tick} {count} {ticker}')
        count+=1
        time.sleep(1.5)
    return excluded_stock

def stock_metadata(ROOT_PATH='./'):
    '''Fetch available (traded)
    stock reference.
    '''
    # Read investing metadata
    metadata_conn = sqlite3.connect(f'{ROOT_PATH}investing_data.db', timeout=10)

    sql = f'select * from metadata'
    investing_metadata = pd.read_sql(sql, metadata_conn)
    return investing_metadata
      
def calculate_stock_volume_contribution(origin_db, target_db, excluded_stock, selected_stock_only=False, ROOT_PATH='./'):
    '''Calculate ratio of traded stock volume
    for each day compared to IDX total volume
    in each day.
    
    Store calculated ratio in new table 'IDX'
    '''
    origin_db_conn = sqlite3.connect(origin_db)
    target_db_conn = sqlite3.connect(target_db)
    
    # Delete all df variables that exists
    try:
        del df
    except NameError:
        pass
    
    investing_metadata = stock_metadata(ROOT_PATH=ROOT_PATH)
    selected_stock = stock_list_100_highestrank_and_availability()
    
    for index in range(len(investing_metadata)):
        ticker1 = investing_metadata.loc[index]['ticker']
        
        if (ticker1 not in selected_stock and selected_stock_only) or ticker1 in excluded_stock:
            continue

        df1 = pd.read_sql(f'select time, Volume from {ticker1}', origin_db_conn)

        try:
            df = df.merge(df1, how='outer')
            df = df.rename(columns={'Volume':ticker1})
        except NameError:
            df = df1.copy(deep=True)
            df = df.rename(columns={'Volume':ticker1})

    # Calculate stock volume
    df_volume = df.copy(deep=True)
    df_volume = df.drop('time', axis='columns')
    df_volume = df_volume.fillna(0)
    df_volume['IDX'] = df_volume.sum(axis=1)

    # df_volume
    df['IDX'] = df_volume['IDX']
    df = df.sort_values(by='time')

    # Calculate the stock volume relative to total volume
    for index in range(len(investing_metadata)):
        ticker1 = investing_metadata.loc[index]['ticker']
        
        if (ticker1 not in selected_stock and selected_stock_only) or ticker1 in excluded_stock:
            continue
        
        df[f'{ticker1}_'] = df[ticker1] / df['IDX']

    # Drop stock traded volume
    df = df.drop([x for x in list(investing_metadata['ticker']) if x not in excluded_stock], axis='columns')
    
    # Drop row that has zero sum of volumes
    df = df[df.IDX != 0.0]

    # Save dataframe into database
    df.to_sql(name='IDX', con=target_db_conn, index=False, if_exists='replace')
        
def calculate_EMA(df, column, emas=(3,10,30,200)):
    for ema in emas:
        df[f'{column}_EMA{ema}'] = df[column].ewm(span=ema, adjust=False).mean()
    return df

# Old version
# calculate gradient from 4 samples
def calculate_EMA_gradient(df, column, emas=(3,10,30,200)):
    for ema in emas:
        multiplier = [1, 1/2, 3/4, 7/8]
        samples = [int(m * ema) for m in multiplier]
        sample_columns = []
        for i, sample in enumerate(samples):
            sample_column_name = f'{column}_EMA{ema}_G{i}'
            sample_columns.append(sample_column_name)
            df[sample_column_name] = (df[f'{column}_EMA{ema}'] - df[f'{column}_EMA{ema}'].shift(sample)) / ema
        df[f'{column}_EMA{ema}_G'] = df.loc[:, sample_columns].sum(axis=1) / len(samples)
        df = df.drop(sample_columns, axis='columns')
    return df

# Revised version
# Calculate gradient from tn-1 only
def calculate_EMA_gradient(df, column, emas=(3,10,30,200)):
    for ema in emas:
        df[f'{column}_EMA{ema}_G'] = df[f'{column}_EMA{ema}'] - df[f'{column}_EMA{ema}'].shift(1)
    return df

def calculate_signal_EMA_offset(df, column, signal=3, emas=(10,30,200)):
    signal_column = f'{column}_EMA{signal}'
    for ema in emas:
        source_column = f'{column}_EMA{ema}'
        target_column = f'{column}_EMA{signal}_EMA{ema}_offset'
        df[target_column] = (df[signal_column] - df[source_column]) / df[source_column]
    return df

def calculate_candle_score(df, columns=('open','high','low','close','change')):
    open_c, high_c, low_c, close_c, change_c = columns
    candle_I0, candle_I1, candle_I2, candle_I3, candle_I4, candle_I5 = ('candle_I0', 'candle_I1', 'candle_I2', 'candle_I3', 'candle_I4', 'candle_I5')
    candle_S1, candle_S2, candle_S3, candle_S4 = ('candle_S1', 'candle_S2', 'candle_S3', 'candle_S4')
    
    # Candle body
    df[candle_I0] = df[open_c] - df[close_c]
    
    # Identify red / green candle status
    df[candle_I1] = np.select([df[candle_I0] < 0, df[candle_I0] > 0, df[candle_I0].isna()],
                              [-1, 1, np.nan], default=0)
    
    # High-low range, relative to close price
    df[candle_I2] = (df[high_c] - df[low_c]) / df[close_c]
    
    # Absolute relative body length to close price
    df[candle_I3] = (df[candle_I0] / df[close_c]).abs()
    
    # Body length / high-low range ratio
    df[candle_I4] = df[candle_I3] / df[candle_I2]
    
    # Candle body offset relative to high-low mean
    df[candle_I5] = ((((df[high_c] - df[low_c]) / 2) + 
                      ((df[close_c] - df[open_c]) / 2)) / 
                     ((df[high_c] - df[low_c]) / 2))
        
    # Score1: product of I1 * sum of I2-I5
    df[candle_S1] = df[candle_I1] * (df[candle_I2] + df[candle_I3] + df[candle_I4] + df[candle_I5])
    
    # Score2: product of I1 * average if I2-I5
    df[candle_S2] = df[candle_I1] * (df[candle_I2] + df[candle_I3] + df[candle_I4] + df[candle_I5]) / 4
    
    # Score3: product of I1-I5
    df[candle_S3] = df[candle_I1] * df[candle_I2] * df[candle_I3] * df[candle_I4] * df[candle_I5]
    
    # Score4: product of I1 * absolute of I2-I5 product
    df[candle_S4] = df[candle_I1] * (df[candle_I2] * df[candle_I3] * df[candle_I4] * df[candle_I5]).abs()
    
    return df
    
def calculate_favorite_stock(FAVORITE_STOCK, DB_PATH, threshold=20, column_groups=('day', 'month', 'year'), ROOT_PATH='./'):
    '''day-month-year favorite
    FAVORITE_STOCK: string, path to save the results.
    '''
    stocks_db_conn = sqlite3.connect(f'{ROOT_PATH}{DB_PATH}')
    df_IDX = pd.read_sql('select * from IDX', stocks_db_conn)

    # Convert integer timestamp into date
    df_IDX['time'] = pd.to_datetime(df_IDX['time'])

    # Make new column for year/month/day
    df_IDX.loc[:,'year'] = df_IDX['time'].dt.year
    df_IDX.loc[:,'month'] = df_IDX['time'].dt.month
    df_IDX.loc[:,'day'] = df_IDX['time'].dt.day

    # Drop time column to avoid interference to the rank
    df_IDX_rank = df_IDX.drop(['time','IDX','day','month','year'], axis='columns')

    # Calculate rank
    df_IDX_rank = df_IDX_rank.rank(axis=1, ascending=True)

    # Normalize rank
    df_IDX_rank = df_IDX_rank.apply(lambda x: x / df_IDX_rank.count(axis=1))

    all_results = {}
    for column_group in column_groups:
        groups_result = {}
        for group_value in df_IDX[column_group].unique():
            tick = datetime.datetime.now()

            # Calculate rank for every specified range
            df_IDX_filtered = df_IDX.loc[df_IDX[column_group] == group_value]
            record_length = len(df_IDX_filtered)
            
            group_result = {}
            weight = np.arange(1, threshold+1)[::-1] / record_length

            # Loop through selected index
            for index in df_IDX_filtered.index:
                top_tickers = df_IDX_rank.iloc[index].sort_values(ascending=False)[:threshold].index


                for i, top_ticker in enumerate(top_tickers):
                    if top_ticker in group_result:
                        group_result[top_ticker] = group_result[top_ticker] + weight[i]
                    else:
                        group_result[top_ticker] = weight[i]

            groups_result[str(group_value)] = group_result

            tock = datetime.datetime.now()
            print(f'{str(column_group)} - {group_value}')

        all_results[column_group] = groups_result
        with open(FAVORITE_STOCK, 'w') as f:
            json.dump(all_results, f)
            
def calculate_favorite_stockv2(FAVORITE_STOCK, DB_PATH, threshold=20, main_group='year', sub_group='day', ROOT_PATH='./'):
    '''day-year / month-year
    FAVORITE_STOCK: string, path to save the results.
    '''
    stocks_db_conn = sqlite3.connect(f'{ROOT_PATH}{DB_PATH}')
    df_IDX = pd.read_sql('select * from IDX', stocks_db_conn)

    # Convert integer timestamp into date
    df_IDX['time'] = pd.to_datetime(df_IDX['time'])

    # Make new column for year/month/day
    df_IDX.loc[:,'year'] = df_IDX['time'].dt.year
    df_IDX.loc[:,'month'] = df_IDX['time'].dt.month
    df_IDX.loc[:,'day'] = df_IDX['time'].dt.day

    # Drop time column to avoid interference to the rank
    df_IDX_rank = df_IDX.drop(['time','IDX','day','month','year'], axis='columns')

    # Calculate rank
    df_IDX_rank = df_IDX_rank.rank(axis=1, ascending=True)

    # Normalize rank
    df_IDX_rank = df_IDX_rank.apply(lambda x: x / df_IDX_rank.count(axis=1))

    groups_result = {}
    for main_value in df_IDX[main_group].unique():
        for sub_value in df_IDX[sub_group].unique():
            tick = datetime.datetime.now()

            # Calculate rank for every specified range
            df_IDX_filtered = df_IDX.loc[(df_IDX[main_group] == main_value) & (df_IDX[sub_group] == sub_value)]
            record_length = len(df_IDX_filtered)

            group_result = {}
            weight = np.arange(1, threshold+1)[::-1] / record_length

            # Loop through selected index
            for index in df_IDX_filtered.index:
                top_tickers = df_IDX_rank.iloc[index].sort_values(ascending=False)[:threshold].index

                for i, top_ticker in enumerate(top_tickers):
                    if top_ticker in group_result:
                        group_result[top_ticker] = group_result[top_ticker] + weight[i]
                    else:
                        group_result[top_ticker] = weight[i]

            groups_result[f"{main_value}_{f'0{sub_value}' if sub_value < 10 else f'{sub_value}'}"] = group_result

            tock = datetime.datetime.now()
            print(f'{main_value} - {sub_value}')

            with open(FAVORITE_STOCK, 'w') as f:
                json.dump(groups_result, f)
                
def calculate_horizontal_support_resistance(df, ticker, indicators):
    '''
    ticker: string
    indicator: dict
    '''
#     dfhs = pd.read_sql(f'select * from {ticker}', stocks_db_conn)
    dfhs = df.copy(deep=True)
    
    sr_lines = {}
    # Calculate horizontal support/resistance lines
    for indicator in indicators:
        # Stack multiple column to single column
        dfhs_sliced = dfhs[indicators[indicator]]
        try:
            bins = int(len(dfhs_sliced.stack()) / 10)

            # Calculate histogram intervals
            x = pd.cut(dfhs_sliced.stack(), bins).value_counts()
        except ValueError:
            bins = len(dfhs_sliced.stack())

            # Calculate histogram intervals
            x = pd.cut(dfhs_sliced.stack(), bins).value_counts()

        # Calculate middle value of interval and store to new df
        mid_values = []
        for mid in x.index:
            mid_values.append((mid.mid, x[mid]))
        hist_pd = pd.DataFrame(mid_values)

        # Filter horizontal s/r lines that has less than mean frequency
        sr_line_df = hist_pd.loc[hist_pd[1] <= hist_pd[1].mean()].sort_values(by=0)
        sr_line_df = sr_line_df.reset_index(drop=True)

        # Get 10 s/r lines with same range between them
        sr_thresholds = np.linspace(0,1,10)
        calculated_sr = []
        for sr_threshold in sr_thresholds:
            calculated_sr.append(sr_line_df.iloc[int((len(sr_line_df) - 1) * sr_threshold)][0])
        sr_lines[indicator] = calculated_sr
    return sr_lines

def __comp__calculate_stock_change_ratio(df, shift=1):
    '''Calculate stock change in %.
    Default shift value is 1, meaning that
    it's calculating daily stock price change.    
    '''
    df['change'] = (df['close'] - df['close'].shift(shift)) / df['close'].shift(shift)
    return df

def __comp__calculate_stock_volume_rank(df, df_IDX_rank, ticker):
    '''Calculate stock volume rank from `IDX` table
    and insert it into individual stock table
    '''
    df = df.merge(df_IDX_rank[['time', f'{ticker}_']], how='inner')
    df = df.rename(columns={f'{ticker}_':'Volume_rank'})
    return df

def __comp__calculate_stock_indicator(df):
    source_columns = ['close', 'rsi14', 'Volume', 'Volume_rank', 'change']
    df['rsi14'] = talib.RSI(df['close'], timeperiod=14)
    df = calculate_candle_score(df)
    for source_column in source_columns:
        df = calculate_EMA(df, source_column)
        df = calculate_EMA_gradient(df, source_column)
        df = calculate_signal_EMA_offset(df, source_column)
    return df

def __comp__calculate_oscillation_between_sr(df, origin_db_conn, ticker):
    indicators = {'close':['open','high','low','close'],'rsi14':['rsi14'],'Volume':['Volume'],'Volume_rank':['Volume_rank'],'change':['change']}
    sr_lines = calculate_horizontal_support_resistance(df, ticker, indicators)

    # Calculate indicator progress between interval
    for indicator in indicators:
        # Define condition and choice list
        condlist = [df[indicator] <= sr for sr in sr_lines[indicator][1:]]
        choicelist_t = [sr for sr in sr_lines[indicator][1:]]
        choicelist_b = [sr for sr in sr_lines[indicator][:len(sr_lines[indicator]) - 1]]
        df[f'{indicator}_b'] = np.select(condlist, choicelist_b, default=choicelist_b[-1])
        df[f'{indicator}_t'] = np.select(condlist, choicelist_t, default=choicelist_t[-1])

        # Calculate progress between interval.
        # *basically, just min/max norm between bottom/top interval
        df[f'{indicator}_srp'] = (df[indicator] - df[f'{indicator}_b']) / (df[f'{indicator}_t'] - df[f'{indicator}_b'])
        df = df.drop([f'{indicator}_b',f'{indicator}_t'], axis='columns')

    # Add close price relative to all time low / high
    df['close_rel'] = (df['close'] - df['close'].min()) / df['close'].max()
    return df

def __comp__calculate_cumulative_change(df):
    '''Calculate price change f_shift ahead from
    previous day.
    '''
    f_shifts=(3,5,7,10)
    b_shifts = [1 for _ in range(len(f_shifts))]
    for i, f_shift in enumerate(f_shifts):
        b_shift = b_shifts[i]
        df[f'change_b{b_shift}f{f_shift}'] = (df['close'].shift(-f_shift) - df['close'].shift(b_shift)) / df['close'].shift(b_shift)
    return df

def __comp__calculate_forecast_column(df):
    '''Calculate close_EMA3_G and close_EMA10_G
    at +1 and +2 forecast
    '''
    columns_to_forecast = ('close_EMA3_G','close_EMA10_G')
    forecast_lengths = (1,2)
    for column_to_forecast in columns_to_forecast:
        for forecast_length in forecast_lengths:
            df[f'{column_to_forecast}_s{forecast_length}'] = df[column_to_forecast].shift(-forecast_length)
    return df

def _calculate_stock_volume_rank(target_db):
    '''Component of calculate_stock_volume_rank    
    '''
    # Fetch portion to IHSG
    stocks_db_conn = sqlite3.connect(target_db)
    df_IDX = pd.read_sql(f'select * from IDX', stocks_db_conn)

    # Drop time column to avoid interference to the rank
    df_IDX_rank = df_IDX.drop(['time','IDX'], axis='columns')

    # Calculate rank
    df_IDX_rank = df_IDX_rank.rank(axis=1, ascending=True)

    # Normalize rank
    df_IDX_rank = df_IDX_rank.apply(lambda x: x / df_IDX_rank.count(axis=1))

    # Bring back time column
    df_IDX_rank.insert(0, 'time', df_IDX['time'])
    return df_IDX_rank

def calculate_all_indicator(origin_db, target_db, excluded_stock, verbose=1, selected_stock_only=False, ROOT_PATH='./'):
    origin_db_conn = sqlite3.connect(origin_db)
    target_db_conn = sqlite3.connect(target_db)
    investing_metadata = stock_metadata(ROOT_PATH=ROOT_PATH)
    count = 0
    
    # calculate_stock_volume_contribution
    # Calculate ratio of traded stock volume for
    # each day compared to IDX total volume each day
    calculate_stock_volume_contribution(origin_db, target_db, excluded_stock, selected_stock_only=selected_stock_only, ROOT_PATH=ROOT_PATH)
    
    # Fetch calculated IDX rank data frame
    df_IDX_rank = _calculate_stock_volume_rank(target_db)
    
    selected_stock = stock_list_100_highestrank_and_availability()
    
    for index in range(len(investing_metadata)):
        ticker = investing_metadata.loc[index]['ticker']
        
        if (ticker not in selected_stock and selected_stock_only) or ticker in excluded_stock:
            continue    
        
        # Read origin stock data
        df = pd.read_sql(f'select * from {ticker}', origin_db_conn)
        
        # Some indicator calculation #
        # calculate_stock_change_ratio
        df = __comp__calculate_stock_change_ratio(df, shift=1)
        
        # calculate_stock_volume_rank
        # Merge volume rank into individual stock df
        df = __comp__calculate_stock_volume_rank(df, df_IDX_rank, ticker)
        
        # calculate_stock_indicator
        df = __comp__calculate_stock_indicator(df)
            
        # calculate_oscillation_between_sr
        df = __comp__calculate_oscillation_between_sr(df, origin_db_conn, ticker)
        
        # calculate_cumulative_change
        df = __comp__calculate_cumulative_change(df)
            
        # calculate_forecast_column
        df = __comp__calculate_forecast_column(df)     
        ##############################
        
        # Write back into table
        df.to_sql(name=ticker, con=target_db_conn, index=False, if_exists='replace')
        
        count+=1
        if verbose and count%50 == 0:
            print(f'Current progress: {count}/{len(investing_metadata)}')
            
# Additional indicators
# Excluded indicators (unclear variable definition)
# MAVP
# Failed with error code2: Bad Parameter (TA_BAD_PARAM): MAMA (f'{target_columns}_mama', f'{target_columns}_fama')

def _universal_indicators(df, target_columns, source_columns):
    close = df[target_columns]
    add_source_columns = [f'{target_columns}_bbands_upper', f'{target_columns}_bbands_middle', f'{target_columns}_bbands_lower', f'{target_columns}_dema', f'{target_columns}_ema', f'{target_columns}_ht_trendline', f'{target_columns}_kama', f'{target_columns}_real', f'{target_columns}_midpoint', f'{target_columns}_sma', f'{target_columns}_t3', f'{target_columns}_tema', f'{target_columns}_trima', f'{target_columns}_wma', f'{target_columns}_apo', f'{target_columns}_cmo', f'{target_columns}_macd', f'{target_columns}_macdsignal', f'{target_columns}_macdhist', f'{target_columns}_macd_ext', f'{target_columns}_macdsignal_ext', f'{target_columns}_macdhist_ext', f'{target_columns}_macd_fix', f'{target_columns}_macdsignal_fix', f'{target_columns}_macdhist_fix', f'{target_columns}_mom', f'{target_columns}_ppo', f'{target_columns}_roc', f'{target_columns}_rocp', f'{target_columns}_rocr', f'{target_columns}_rocr100', f'{target_columns}_rsi', f'{target_columns}_stochrsi_fastk', f'{target_columns}_stochrsi_fastd', f'{target_columns}_trix', f'{target_columns}_ht_dcperiod', f'{target_columns}_ht_dcphase', f'{target_columns}_ht_phasor_inphase', f'{target_columns}_ht_phasor_quadrature', f'{target_columns}_ht_sine_sine', f'{target_columns}_ht_sine_leadsine', f'{target_columns}_ht_trendmode']
    for i in add_source_columns: source_columns.append(i)
    # Overlap Studies Functions
    df[f'{target_columns}_bbands_upper'], df[f'{target_columns}_bbands_middle'], df[f'{target_columns}_bbands_lower'] = talib.BBANDS(close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    df[f'{target_columns}_dema'] = talib.DEMA(close, timeperiod=30)
    df[f'{target_columns}_ema'] = talib.EMA(close, timeperiod=30)
    df[f'{target_columns}_ht_trendline'] = talib.HT_TRENDLINE(close)
    df[f'{target_columns}_kama'] = talib.KAMA(close, timeperiod=30)
    df[f'{target_columns}_real'] = talib.MA(close, timeperiod=30, matype=0)
    # df[f'{target_columns}_mama'], df[f'{target_columns}_fama'] = talib.MAMA(close, fastlimit=0, slowlimit=0)
    df[f'{target_columns}_midpoint'] = talib.MIDPOINT(close, timeperiod=14)
    df[f'{target_columns}_sma'] = talib.SMA(close, timeperiod=30)
    df[f'{target_columns}_t3'] = talib.T3(close, timeperiod=5, vfactor=0)
    df[f'{target_columns}_tema'] = talib.TEMA(close, timeperiod=30)
    df[f'{target_columns}_trima'] = talib.TRIMA(close, timeperiod=30)
    df[f'{target_columns}_wma'] = talib.WMA(close, timeperiod=30)
    
    # Momentum indicator functions
    df[f'{target_columns}_apo'] = talib.APO(close, fastperiod=12, slowperiod=26, matype=0)
    df[f'{target_columns}_cmo'] = talib.CMO(close, timeperiod=14)
    df[f'{target_columns}_macd'], df[f'{target_columns}_macdsignal'], df[f'{target_columns}_macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    df[f'{target_columns}_macd_ext'], df[f'{target_columns}_macdsignal_ext'], df[f'{target_columns}_macdhist_ext'] = talib.MACDEXT(close, fastperiod=12, fastmatype=0, slowperiod=26, slowmatype=0, signalperiod=9, signalmatype=0)
    df[f'{target_columns}_macd_fix'], df[f'{target_columns}_macdsignal_fix'], df[f'{target_columns}_macdhist_fix'] = talib.MACDFIX(close, signalperiod=9)
    df[f'{target_columns}_mom'] = talib.MOM(close, timeperiod=10)
    df[f'{target_columns}_ppo'] = talib.PPO(close, fastperiod=12, slowperiod=26, matype=0)
    df[f'{target_columns}_roc'] = talib.ROC(close, timeperiod=10)
    df[f'{target_columns}_rocp'] = talib.ROCP(close, timeperiod=10)
    df[f'{target_columns}_rocr'] = talib.ROCR(close, timeperiod=10)
    df[f'{target_columns}_rocr100'] = talib.ROCR100(close, timeperiod=10)
    df[f'{target_columns}_rsi'] = talib.RSI(close, timeperiod=14)
    df[f'{target_columns}_stochrsi_fastk'], df[f'{target_columns}_stochrsi_fastd'] = talib.STOCHRSI(close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
    df[f'{target_columns}_trix'] = talib.TRIX(close, timeperiod=30)
    
    # Cycle indicator functions
    df[f'{target_columns}_ht_dcperiod'] = talib.HT_DCPERIOD(close)
    df[f'{target_columns}_ht_dcphase'] = talib.HT_DCPHASE(close)
    df[f'{target_columns}_ht_phasor_inphase'], df[f'{target_columns}_ht_phasor_quadrature'] = talib.HT_PHASOR(close)
    df[f'{target_columns}_ht_sine_sine'], df[f'{target_columns}_ht_sine_leadsine'] = talib.HT_SINE(close)
    df[f'{target_columns}_ht_trendmode'] = talib.HT_TRENDMODE(close)
    return df, source_columns

def _price_indicators(df, source_columns):
    open = df['open']
    high = df['high']
    low = df['low']
    close = df['close']
    volume = df['Volume']
    add_source_columns = ['midprice', 'sar', 'sarext', 'adx', 'adxr', 'aroondown', 'aroonup', 'aroonosc', 'bop', 'cci', 'dx', 'mfi', 'minus_di', 'minus_dm', 'plus_di', 'plus_dm', 'stoch_slowk', 'stoch_slowd', 'stochf_fastk', 'stochf_fastd', 'ultosc', 'willr', 'ad', 'adosc', 'obv', 'avgprice', 'medprice', 'typprice', 'wclprice', 'atr', 'natr', 'trange']
    for i in add_source_columns: source_columns.append(i)
    # Overlap Studies Functions
    df['midprice'] = talib.MIDPRICE(high, low, timeperiod=14)
    df['sar'] = talib.SAR(high, low, acceleration=0, maximum=0)
    df['sarext'] = talib.SAREXT(high, low, startvalue=0, offsetonreverse=0, accelerationinitlong=0, accelerationlong=0, accelerationmaxlong=0, accelerationinitshort=0, accelerationshort=0, accelerationmaxshort=0)
    
    # Momentum indicator functions
    df['adx'] = talib.ADX(high, low, close, timeperiod=14)
    df['adxr'] = talib.ADXR(high, low, close, timeperiod=14)
    df['aroondown'], df['aroonup'] = talib.AROON(high, low, timeperiod=14)
    df['aroonosc'] = talib.AROONOSC(high, low, timeperiod=14)
    df['bop'] = talib.BOP(open, high, low, close)
    df['cci'] = talib.CCI(high, low, close, timeperiod=14)
    df['dx'] = talib.DX(high, low, close, timeperiod=14)
    df['mfi'] = talib.MFI(high, low, close, volume, timeperiod=14)
    df['minus_di'] = talib.MINUS_DI(high, low, close, timeperiod=14)
    df['minus_dm'] = talib.MINUS_DM(high, low, timeperiod=14)
    df['plus_di'] = talib.PLUS_DI(high, low, close, timeperiod=14)
    df['plus_dm'] = talib.PLUS_DM(high, low, timeperiod=14)
    df['stoch_slowk'], df['stoch_slowd'] = talib.STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    df['stochf_fastk'], df['stochf_fastd'] = talib.STOCHF(high, low, close, fastk_period=5, fastd_period=3, fastd_matype=0)
    df['ultosc'] = talib.ULTOSC(high, low, close, timeperiod1=7, timeperiod2=14, timeperiod3=28)
    df['willr'] = talib.WILLR(high, low, close, timeperiod=14)
    
    # Volume indicator functions
    df['ad'] = talib.AD(high, low, close, volume)
    df['adosc'] = talib.ADOSC(high, low, close, volume, fastperiod=3, slowperiod=10)
    df['obv'] = talib.OBV(close, volume)
    
    # Price transform functions
    df['avgprice'] = talib.AVGPRICE(open, high, low, close)
    df['medprice'] = talib.MEDPRICE(high, low)
    df['typprice'] = talib.TYPPRICE(high, low, close)
    df['wclprice'] = talib.WCLPRICE(high, low, close)
    
    # Volatility indicator functions
    df['atr'] = talib.ATR(high, low, close, timeperiod=14)
    df['natr'] = talib.NATR(high, low, close, timeperiod=14)
    df['trange'] = talib.TRANGE(high, low, close)
    return df, source_columns

# Pattern recognition functions
def _pattern_recognition(df, source_columns):
    open = df['open']
    high = df['high']
    low = df['low']
    close = df['close']
    add_source_columns = ['cdl2crows', 'cdl3blackrows', 'cdl3inside', 'cdl3linestrike', 'cdl3outside', 'cdl3starsinsouth', 'cdl3whitesoldiers', 'cdlabandonedbaby', 'cdladvanceblock', 'cdlbelthold', 'cdlbreakaway', 'cdlclosingmarubozu', 'cdlconcealbabyswall', 'cdlcounterattack', 'cdldarkcloudcover', 'cdldoji', 'cdldojistar', 'cdldragonflydoji', 'cdlengulfing', 'cdleveningdojistar', 'cdleveningstar', 'cdlgapinsidewhite', 'cdlgravestonedoji', 'cdlhammer', 'cdlhangingman', 'cdlharami', 'cdlharamicross', 'cdlhighwave', 'cdlhikkake', 'cdlhikkakemod', 'cdlhomingpigeon', 'cdlidentical3crows', 'cdlinneck', 'cdlinvertedhammer', 'cdlkicking', 'cdlkickingbylength', 'cdlladderbottom', 'cdllongleggeddoji', 'cdllongline', 'cdlmarubozu', 'cdlmatchinglow', 'cdlmathold', 'cdlmorningdojistar', 'cdlmorningstar', 'cdlonneck', 'cdlpiercing', 'cdlrickshawman', 'cdlrisefall3methods', 'cdlseparatinglines', 'cdlshootingstar', 'cdlshortline', 'cdlspinningtop', 'cdlstalledpattern', 'cdlsticksandwich', 'cdltakuri', 'cdltasukigap', 'cdlthrusting', 'cdltristar', 'cdlunique3river', 'cdlupsidegap2crows', 'cdlxsidegap3methods']
    for i in add_source_columns: source_columns.append(i)
    df['cdl2crows'] = talib.CDL2CROWS(open, high, low, close)
    df['cdl3blackrows'] = talib.CDL3BLACKCROWS(open, high, low, close)
    df['cdl3inside'] = talib.CDL3INSIDE(open, high, low, close)
    df['cdl3linestrike'] = talib.CDL3LINESTRIKE(open, high, low, close)
    df['cdl3outside'] = talib.CDL3OUTSIDE(open, high, low, close)
    df['cdl3starsinsouth'] = talib.CDL3STARSINSOUTH(open, high, low, close)
    df['cdl3whitesoldiers'] = talib.CDL3WHITESOLDIERS(open, high, low, close)
    df['cdlabandonedbaby'] = talib.CDLABANDONEDBABY(open, high, low, close, penetration=0)
    df['cdladvanceblock'] = talib.CDLADVANCEBLOCK(open, high, low, close)
    df['cdlbelthold'] = talib.CDLBELTHOLD(open, high, low, close)
    df['cdlbreakaway'] = talib.CDLBREAKAWAY(open, high, low, close)
    df['cdlclosingmarubozu'] = talib.CDLCLOSINGMARUBOZU(open, high, low, close)
    df['cdlconcealbabyswall'] = talib.CDLCONCEALBABYSWALL(open, high, low, close)
    df['cdlcounterattack'] = talib.CDLCOUNTERATTACK(open, high, low, close)
    df['cdldarkcloudcover'] = talib.CDLDARKCLOUDCOVER(open, high, low, close, penetration=0)
    df['cdldoji'] = talib.CDLDOJI(open, high, low, close)
    df['cdldojistar'] = talib.CDLDOJISTAR(open, high, low, close)
    df['cdldragonflydoji'] = talib.CDLDRAGONFLYDOJI(open, high, low, close)
    df['cdlengulfing'] = talib.CDLENGULFING(open, high, low, close)
    df['cdleveningdojistar'] = talib.CDLEVENINGDOJISTAR(open, high, low, close, penetration=0)
    df['cdleveningstar'] = talib.CDLEVENINGSTAR(open, high, low, close, penetration=0)
    df['cdlgapinsidewhite'] = talib.CDLGAPSIDESIDEWHITE(open, high, low, close)
    df['cdlgravestonedoji'] = talib.CDLGRAVESTONEDOJI(open, high, low, close)
    df['cdlhammer'] = talib.CDLHAMMER(open, high, low, close)
    df['cdlhangingman'] = talib.CDLHANGINGMAN(open, high, low, close)
    df['cdlharami'] = talib.CDLHARAMI(open, high, low, close)
    df['cdlharamicross'] = talib.CDLHARAMICROSS(open, high, low, close)
    df['cdlhighwave'] = talib.CDLHIGHWAVE(open, high, low, close)
    df['cdlhikkake'] = talib.CDLHIKKAKE(open, high, low, close)
    df['cdlhikkakemod'] = talib.CDLHIKKAKEMOD(open, high, low, close)
    df['cdlhomingpigeon'] = talib.CDLHOMINGPIGEON(open, high, low, close)
    df['cdlidentical3crows'] = talib.CDLIDENTICAL3CROWS(open, high, low, close)
    df['cdlinneck'] = talib.CDLINNECK(open, high, low, close)
    df['cdlinvertedhammer'] = talib.CDLINVERTEDHAMMER(open, high, low, close)
    df['cdlkicking'] = talib.CDLKICKING(open, high, low, close)
    df['cdlkickingbylength'] = talib.CDLKICKINGBYLENGTH(open, high, low, close)
    df['cdlladderbottom'] = talib.CDLLADDERBOTTOM(open, high, low, close)
    df['cdllongleggeddoji'] = talib.CDLLONGLEGGEDDOJI(open, high, low, close)
    df['cdllongline'] = talib.CDLLONGLINE(open, high, low, close)
    df['cdlmarubozu'] = talib.CDLMARUBOZU(open, high, low, close)
    df['cdlmatchinglow'] = talib.CDLMATCHINGLOW(open, high, low, close)
    df['cdlmathold'] = talib.CDLMATHOLD(open, high, low, close, penetration=0)
    df['cdlmorningdojistar'] = talib.CDLMORNINGDOJISTAR(open, high, low, close, penetration=0)
    df['cdlmorningstar'] = talib.CDLMORNINGSTAR(open, high, low, close, penetration=0)
    df['cdlonneck'] = talib.CDLONNECK(open, high, low, close)
    df['cdlpiercing'] = talib.CDLPIERCING(open, high, low, close)
    df['cdlrickshawman'] = talib.CDLRICKSHAWMAN(open, high, low, close)
    df['cdlrisefall3methods'] = talib.CDLRISEFALL3METHODS(open, high, low, close)
    df['cdlseparatinglines'] = talib.CDLSEPARATINGLINES(open, high, low, close)
    df['cdlshootingstar'] = talib.CDLSHOOTINGSTAR(open, high, low, close)
    df['cdlshortline'] = talib.CDLSHORTLINE(open, high, low, close)
    df['cdlspinningtop'] = talib.CDLSPINNINGTOP(open, high, low, close)
    df['cdlstalledpattern'] = talib.CDLSTALLEDPATTERN(open, high, low, close)
    df['cdlsticksandwich'] = talib.CDLSTICKSANDWICH(open, high, low, close)
    df['cdltakuri'] = talib.CDLTAKURI(open, high, low, close)
    df['cdltasukigap'] = talib.CDLTASUKIGAP(open, high, low, close)
    df['cdlthrusting'] = talib.CDLTHRUSTING(open, high, low, close)
    df['cdltristar'] = talib.CDLTRISTAR(open, high, low, close)
    df['cdlunique3river'] = talib.CDLUNIQUE3RIVER(open, high, low, close)
    df['cdlupsidegap2crows'] = talib.CDLUPSIDEGAP2CROWS(open, high, low, close)
    df['cdlxsidegap3methods'] = talib.CDLXSIDEGAP3METHODS(open, high, low, close)
    return df, source_columns

def _indicator_derivatives(df, source_columns):
    # Calculate EMA, their gradient, and signal offset for every indicator
    for source_column in source_columns:
        df = calculate_EMA(df, source_column)
        df = calculate_EMA_gradient(df, source_column)
        df = calculate_signal_EMA_offset(df, source_column)
    return df

def calculate_talib_indicators_primary(df):
    '''Without gradient and their derivatives
    without candle pattern
    '''
    source_columns = []
    # Calculate universal_indicators for every target columns
    target_columns = ['open','high','low','close','Volume','change','Volume_rank']

    for target_column in target_columns:
        df, source_columns = _universal_indicators(df, target_column, source_columns)
    # Calculate price_indicators for price
    df, source_columns = _price_indicators(df, source_columns)
    return df, source_columns

def calculate_talib_indicators(df):
    source_columns = []
    # Calculate universal_indicators for every target columns
    target_columns = ['open','high','low','close','Volume','change','Volume_rank']

    for target_column in target_columns:
        df, source_columns = _universal_indicators(df, target_column, source_columns)
    # Calculate price_indicators for price
    df, source_columns = _price_indicators(df, source_columns)
    # Recognize candle pattern
    df, source_columns = _pattern_recognition(df, source_columns)

    # Calculate EMA, their gradient, and signal offset for every indicator
    df = _indicator_derivatives(df, source_columns)
    return df

def store_splitdf(df, primary_index, conn, max_column=500):
    column_length = len(df.columns)
    splits = math.ceil(column_length / max_column)
    for split in range(splits):
        columns = df.columns[max_column * split:max_column * (1 + split)]
        columns = np.insert(columns, 0, primary_index) if primary_index not in columns else columns
        temp_df = df[columns]
        temp_df.to_sql(str(split), conn, if_exists='replace', index=False)
    
def tablename_list(conn):
    cursor = conn.cursor()
    cursor.execute(f"select name from sqlite_master where type='table'")
    return [x[0] for x in cursor.fetchall()]

def restore_splitdf(conn):
    table_names = tablename_list(conn)
    first = True
    for table_name in table_names:
        temp_df = pd.read_sql(f'select * from `{table_name}`', conn)
        if first:
            df = temp_df.copy(deep=True)
            first = False
        elif not first:
            df = df.merge(temp_df)
    return df

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), 'df needs to be a pd.DataFrame'
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep]

def calculate_indicator_correlation(df):
    primary_index = 'time'
    # Standard score normalization
    df = (df - df.mean()) / df.std()

    columns = df.columns
    results = {}
    combinations = []
    count = 0
    tick = datetime.datetime.now()
    for i in columns:
        for j in columns:
            combination = [i,j]
            if (i == primary_index) or (j == primary_index) or (i == j) or (combination in combinations) or (combination[::-1] in combinations):
                continue
            combinations.append(combination)

            slice_df = clean_dataset(df[[i,j]])

            try:
                results[f'{i},{j}'] = r2_score(slice_df[i], slice_df[j])
            except ValueError:
                continue
        count+=1
    return results

def uncorrelated_indicators(results, minimum=20, maximum=30, interval=0.001, increment=0.00005):
    rdf = pd.DataFrame({'keys':results.keys(), 'values':results.values()})
    len_unique_columns = 0
    while (len_unique_columns < minimum) or (len_unique_columns > maximum):
        # Conditions
        if len_unique_columns < minimum:
            interval+=increment
        elif len_unique_columns > maximum:
            interval-=increment

        # Get slice
        zero_corr_df = rdf.loc[(rdf['values'] <= +interval) & (rdf['values'] >= -interval)]

        # Put value and avoid duplicate
        no_corr_combs = zero_corr_df['keys'].values
        unique_columns = []
        for no_corr_comb in no_corr_combs:
            a, b = no_corr_comb.split(',')
            if a not in unique_columns:
                unique_columns.append(a) 
            if b not in unique_columns:
                unique_columns.append(b) 
        len_unique_columns = len(unique_columns)
    return unique_columns

def calculate_uncorrelated_indicators_db_100stocks(ROOT_PATH='./', db_ver='v1', calculate_v1_indicators=True):
    '''Each DB contains 1 ticker data.
    - v1 indicators + additional primary indicators filtered using correlation coefficient
    - 20-30 most uncorrelated indicators and their derivatives stored into .db
    - close_EMA3 / close_EMA10 derivatives as required columns
    - Max column per table: {selected_stock_only}
    '''
    excluded_stock = excluded_stock()
    required_columns = ['close_EMA3_G', 'close_EMA10_G', 'close_EMA3_G_s1', 'close_EMA3_G_s2', 'close_EMA10_G_s1', 'close_EMA10_G_s2']
    primary_index = 'time'
    max_column_per_table = 250
    selected_stock_only = False

    DB_ROOT_PATH = f'{ROOT_PATH}db/{db_ver}/'
    Path(DB_ROOT_PATH).mkdir(parents=True, exist_ok=True)
    db_readme = f'''DB {db_ver}

    Each DB contains 1 ticker data.
    - v1 indicators + additional primary indicators filtered using correlation coefficient
    - 20-30 most uncorrelated indicators and their derivatives stored into .db
    - close_EMA3 / close_EMA10 derivatives as required columns
    - Max column per table: {selected_stock_only}
    '''
    with open(f'{DB_ROOT_PATH}readme.txt', 'w') as f:
        f.write(db_readme)

    # New flow
    origin_db = f'{ROOT_PATH}idx_raw.db'
    v1_db = f'{ROOT_PATH}idx_indicators.db'
    v2_db_placeholder = '{}idx_{}.db'

    origin_db_conn = sqlite3.connect(origin_db)
    v1_db_conn = sqlite3.connect(v1_db)

    # Calculate v1 indicators
    if calculate_v1_indicators:
        calculate_all_indicator(origin_db, v1_db, excluded_stock, verbose=1, selected_stock_only=selected_stock_only, ROOT_PATH=ROOT_PATH)
    elif not calculate_v1_indicators:
        pass

    tickers = stock_list_100_highestrank_and_availability()

    # Try to resume from previous progress if any
    try:
        with open(f'{DB_ROOT_PATH}progress.cache', 'r') as f:
            hotstart = int(f.read())
    except FileNotFoundError:
        hotstart = 0

    count = 0
    for ticker in tickers:
        if count < hotstart:
            count+=1
            continue
        if ticker in excluded_stock:
            continue
        tick = datetime.datetime.now()
        v2_db = v2_db_placeholder.format(DB_ROOT_PATH, ticker)
        v2_db_conn = sqlite3.connect(v2_db)

        v1_df = pd.read_sql(f'select * from {ticker}', v1_db_conn)

        # Calculate primary indicators
        prim_df, source_columns = calculate_talib_indicators_primary(v1_df)

        # Calculate indicator correlation
        results = calculate_indicator_correlation(prim_df)

        # Eliminate most uncorrelated indicators (20-30 indicators)
        unique_columns = uncorrelated_indicators(results)

        # Check if required_columns are in the unique_columns list, else, append those required_columns
        # close_EMA3_G, close_EMA10_G, close_EMA3_G_s1, close_EMA3_G_s2, close_EMA10_G_s1, close_EMA10_G_s2 are in the list
        for required_column in required_columns:
            if required_column not in unique_columns:
                unique_columns.append(required_column)

        # Add primary_index column at the beginning
        unique_columns.insert(0, primary_index)

        # Add pattern recognition
        prim_df, unique_columns = _pattern_recognition(prim_df, unique_columns)

        # Slice dataframe by unique_columns
        unique_df = prim_df[unique_columns]

        # Calculate indicator derivatives for additional calculated indicator in source_columns
        derivatives_columns = []
        for unique_column in unique_columns:
            if unique_column in source_columns:
                derivatives_columns.append(unique_column)
        derivatives_df = _indicator_derivatives(unique_df, derivatives_columns)

        # Store calculated df as separated .db for each stock
        # with 0->inf. as table name for specified split value
        store_splitdf(derivatives_df, primary_index, v2_db_conn, max_column=max_column_per_table)

        count+=1
        tock = datetime.datetime.now()
        print(f'{count}/{len(tickers)}: {ticker} -- Elapsed time: {tock-tick}')
        with open(f'{DB_ROOT_PATH}progress.cache', 'w') as f:
            f.write(str(count))
            
def load_ml_database(DB_PATH, columns_to_drop=['time']):
    '''Instead of regular read_sql,
    the ml-specialized df need to fill
    null value and drop unnecessary column
    '''
    db_conn = sqlite3.connection(DB_PATH)
    df = restore_splitdf(db_conn)
    df = df.fillna(0)
    for column_to_drop in columns_to_drop:
        try:
            df = df.drop(column_to_drop, axis='columns')
        except ValueError:
            pass
    return df

def input_c_from_sa_v2(ROOT_PATH, model_version, ticker, output_c):
    '''Fetch input_c list from
    sensitivity analysis file
    
    Revision from v1:
    - Revising path to grouping based on 
        simulation version.
    '''
    filename = f'{ROOT_PATH}statistics/v{model_version}/sa_{ticker}_{output_c}.json'

    with open(filename, 'r') as f:
        sa_dict = json.load(f)

    final_column = sorted(sa_dict.items(), key=lambda x: x[1])[0][0]

    input_c = []
    for sa_key in sa_dict.keys():
        input_c.append(sa_key)
        if sa_key == final_column:
            break
    return input_c

def split_traintest(df, split=0.8):
    '''Splif df with train fraction.
    Normalize data using train set mean-std normalization.
    Return normalized train-test set, mean, and std.
    '''
    # Train/test split
    train_df = df[:int(split*len(df))]
    test_df = df[int(split*len(df)):]
    test_df.reset_index(inplace=True, drop=True)

    train_mean = train_df.mean()
    train_std = train_df.std()

    # Normalize dataset
    train_df = (train_df - train_mean) / train_std
    test_df = (test_df - train_mean) / train_std
    return train_df, test_df, train_mean, train_std

def model_compiler(model):
    '''Adam optimizer, mse loss, RMSE metrics
    '''
    model.compile(optimizer='adam',
                  loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

def simpleRNN_1layer(units, return_sequences=False, create_input_layer=False, shape=None):
    if create_input_layer:
        model = tf.keras.models.Sequential([
            tf.keras.Input(shape),
            tf.keras.layers.SimpleRNN(units, return_sequences=return_sequences),
            tf.keras.layers.Dense(1),
        ])
    else:
        model = tf.keras.models.Sequential([
            tf.keras.layers.SimpleRNN(units, return_sequences=return_sequences),
            tf.keras.layers.Dense(1),
        ])
    return model_compiler(model)

def GRU_1layer(units, return_sequences=False, create_input_layer=False, shape=None):
    if create_input_layer:
        model = tf.keras.models.Sequential([
            tf.keras.Input(shape),
            tf.keras.layers.GRU(units, return_sequences=return_sequences),
            tf.keras.layers.Dense(1),
        ])
    else:
        model = tf.keras.models.Sequential([
            tf.keras.layers.GRU(units, return_sequences=return_sequences),
            tf.keras.layers.Dense(1),
        ])
    return model_compiler(model)

def LSTM_1layer(units, return_sequences=False, create_input_layer=False, shape=None):
    if create_input_layer:
        model = tf.keras.models.Sequential([
            tf.keras.Input(shape),
            tf.keras.layers.LSTM(units, return_sequences=return_sequences),
            tf.keras.layers.Dense(1),
        ])
    else:
        model = tf.keras.models.Sequential([
            tf.keras.layers.LSTM(units, return_sequences=return_sequences),
            tf.keras.layers.Dense(1),
        ])
    return model_compiler(model)

def read_optimal_model_config(statistics_file_path, indicator='test_rmse'):
    '''Read most optimal architecture, recurrent
    lstmu, and epoch from optimization simulation.
    '''
    columns = ('ticker','input_c','output_c','architecture','recurrent','lstmu','epoch','train_mse','train_rmse','test_mse','test_rmse')
    sa_DF = pd.read_csv(statistics_file_path, header=0, names=columns)
    best_config = sa_DF.sort_values(by='test_rmse').values[0]
    return best_config[3], best_config[4], best_config[5], best_config[6]

def execute_input_c_v2(ticker, ROOT_PATH, output_c='close_EMA3_G_s1', model_version='2', input_c_limit=25, epochs=(15,), rnnus=(14,), recurrents=(7,), train=0.8):
    '''Performing sensitivity input_c sensitivity analysis
    that satisfies passed parameters
    
    Revision from v1:
    - fixing results/temporary directory confusion
        when running a new version. Grouping results by
        `model_version` params.
    - Fixing output_c temporary fixing that previously
        think that the original EMAx_G is already a +1
        offset value.
    - Replace data loading with `restore_splitdf` function
    
    Important directories
    ./db -> source stock database
    ./statistics -> model performance histories
    '''    
    DB_PATH = f'{ROOT_PATH}db/v{model_version}/idx_{ticker}.db'
    STATISTICS_FILE = f'{ROOT_PATH}statistics/v{model_version}/sa_{ticker}_{output_c}.csv'
    
    # Excluded columns
    excluded_columns_res = ('change_b.f.', '_s[0-9]*')

    # Load data
    df = load_ml_database(DB_PATH)
    
    # Split train-test
    train_df, test_df, train_mean, train_std = split_traintest(df, split=train)

    # Convert all possible input column into dictionary
    # as `additional_c`
    additional_c = [x for x in df.columns]

    # Delete columns that satisfy `excluded_columns_re`
    for excluded_columns_re in excluded_columns_res:
        additional_c = [x for x in additional_c if not re.search(excluded_columns_re, x)]

    # Replace csv with plain statistics file name
    sa_statistics = STATISTICS_FILE.replace('.csv', '')

    try:
        # Fetch `input_s` as hotstart
        # Loop through `additional_c`, and delete keys that have been simulated before
        # in `additional_c`. Using condition in `input_s.keys()`
        with open(f'{sa_statistics}.json', 'r') as f:
            input_s = json.load(f)

        # Delete `additional_c` that have been simulated before
        additional_c = [x for x in additional_c if x not in input_s.keys()]

    except FileNotFoundError:
        input_s = {}

    # Do a simulation as long as there are more `additional_c` to simulate
    # Loop through `additional_c` everytime, until no more left.
    # Better executed with while
    while len(additional_c) > 0 and len(input_s.keys()) < input_c_limit:
        # Use `additional_c_results` dictionary as `additional_c` sim results control. 
        # the format: {add_c: RMSE_performance}
        additional_c_results = {}

        current_ac_loop = 1
        for ac in additional_c:
            # Add additional simulated column into previous input order
            # input_c = [x for x in input_s.keys()]
            # input_c.append(ac)
            # This `input_c` resetted in every `additional_c` loop

            # Configure input column by adding +1 column to a previous
            # configuration to see how much the performance increasing
            input_c = [x for x in input_s.keys()]
            input_c.append(ac)

            for recurrent in recurrents:
                for rnnu in rnnus:
                    for epoch in epochs:
                        tick = datetime.datetime.now()

                        # Make dataset
                        train_dataset = TimeseriesGenerator(train_df[input_c], train_df[output_c], length=recurrent, shuffle=True)
                        test_dataset = TimeseriesGenerator(test_df[input_c], test_df[output_c], length=recurrent)

                        # RNN Model
                        model = GRU_1layer(rnnu, return_sequences=False)
                        
                        # Feed and train the model
                        model.fit(train_dataset, epochs=epoch, verbose=0)

                        # Evaluate model
                        train_eval = model.evaluate(train_dataset, verbose=0)
                        test_eval = model.evaluate(test_dataset, verbose=0)

                        input_c_joined = '|'.join(input_c)
                        statistics = f'''{ticker},{input_c_joined},{output_c},{recurrent},{rnnu},{epoch},{train_eval[0]},{train_eval[1]},{test_eval[0]},{test_eval[1]}\n'''
                        with open(STATISTICS_FILE, 'a') as f:
                            f.write(statistics)

                        tock = datetime.datetime.now()

                        print(f'''{ticker} i[{len(input_c)}] o[{output_c}] acleft[{current_ac_loop}/{len(additional_c)}] aic[{ac}] R[{recurrent}] RU[{rnnu}] E[{epoch}] trainRMSE[{round(train_eval[1], 4)}] testRMSE[{round(test_eval[1], 4)}] time: {tock-tick}''')
                        current_ac_loop+=1

                        # Add testRMSE into `additional_c_results`
                        additional_c_results[ac] = test_eval[1]

        # Everytime the loop through `additional_c` finish,
        # sort the `additional_c_results` using
        # max_res = sorted(additional_c_results.items(), keys=lambda x: x[1])
        max_res = sorted(additional_c_results.items(), key=lambda x: x[1])

        # Fetch max_res[0] to `input_s`
        # input_s[max_res[0][0]] = max_res[0][1]
        input_s[max_res[0][0]] = max_res[0][1]

        # Everytime the SA period finish, the most sensitive
        # additional column deleted from `additional_c`
        additional_c.remove(max_res[0][0])

        print(f'=== MOST SENSITIVE: {max_res[0][0]} with combined RMSE: {max_res[0][1]} ===')

        # Save current `input_s` to file
        with open(f'{sa_statistics}.json', 'w') as f:
            json.dump(input_s, f)
            
def execute_configuration_optimization_v2(ticker, ROOT_PATH, output_c='close_EMA3_G_s1', model_version='2', epochs=(10,20,40), rnnus=(4,8,12,16), recurrents=(5,7,10,15,20), train=0.8):
    '''
    Differences with v1:
    - remove offset parameter, integerating it with 'correct'
        output_c instead of manipulating it on the go. 
    '''
    
    DB_PATH = f'{ROOT_PATH}db/v{model_version}/idx_{ticker}.db'
    STATISTICS_FILE = f'{ROOT_PATH}statistics/v{model_version}/sa_{ticker}_{output_c}.csv'
    input_c = input_c_from_sa_v2(ROOT_PATH, model_version, ticker, output_c)
    architectures = ('SimpleRNN', 'GRU', 'LSTM')

    # Load data
    df = load_ml_database(DB_PATH)
    
    # Split and normalize train-test set
    train_df, test_df, train_mean, train_std = split_traintest(df, split=train)
    
    # Create progress cache
    progress_cache_file = f'{ROOT_PATH}progress_cache/v{model_version}/opt_{ticker}_{output_c}.txt'
    try:
        with open(progress_cache_file, 'r') as f:
            HOTSTART = int(f.read())
    except FileNotFoundError:
        HOTSTART = 0

    loops = 0
    for architecture in architectures:
        for recurrent in recurrents:
            for rnnu in rnnus:
                for epoch in epochs:
                    tick = datetime.datetime.now()
                    # Hotstart
                    if loops < HOTSTART:
                        loops+=1
                        continue

                    # Make dataset
                    train_dataset = TimeseriesGenerator(train_df[input_c], train_df[output_c], length=recurrent, shuffle=True)
                    test_dataset = TimeseriesGenerator(test_df[input_c], test_df[output_c], length=recurrent)

                    # RNN Model
                    if architecture == 'SimpleRNN':
                        model = simpleRNN_1layer(rnnu, return_sequences=False)
                    elif architecture == 'GRU':
                        model = GRU_1layer(rnnu, return_sequences=False)
                    elif architecture == 'LSTM':
                        model = LSTM_1layer(rnnu, return_sequences=False)

                    # Feed and train the model
                    model.fit(train_dataset, epochs=epoch, verbose=0)

                    # Evaluate model
                    train_eval = model.evaluate(train_dataset, verbose=0)
                    test_eval = model.evaluate(test_dataset, verbose=0)

                    input_c_joined = '|'.join(input_c)
                    statistics = f'''{ticker},{input_c_joined},{output_c},{architecture},{recurrent},{rnnu},{epoch},{train_eval[0]},{train_eval[1]},{test_eval[0]},{test_eval[1]}\n'''
                    with open(STATISTICS_FILE, 'a') as f:
                        f.write(statistics)

                    tock = datetime.datetime.now()

                    print(f'''{loops} {ticker} {len(input_c)} {architecture} o[{output_c}] R[{recurrent}] RU[{rnnu}] E[{epoch}] trainRMSE[{round(train_eval[1], 5)}] testRMSE[{round(test_eval[1], 5)}] time: {tock-tick}''')

                    loops+=1
                    # Save progress to cache
                    with open(progress_cache_file, 'w') as f:
                        f.write(str(loops))
            
def execute_batch_v2(simtype, offset, instance_no, ROOT_PATH, split=5, output_cs=('close_EMA3_G_s1','close_EMA10_G_s1'), model_version='2', input_c_limit=25, epochs=(10,), rnnus=(14,), recurrents=(5,), train=0.8):
    '''
    type: string
        input_c
        opt
        
    Revision compared to v1:
    - add excluded stock
    - small revision to adapt to newer `input_c` and `opt` version
    '''
    # Create progress cache
    progress_cache_file = f'{ROOT_PATH}progress_cache/v{model_version}/{simtype}_{offset}_{instance_no}.txt'
    try:
        with open(progress_cache_file, 'r') as f:
            HOTSTART = int(f.read())
    except FileNotFoundError:
        HOTSTART = 0
    
    # Fetch stock metadata
    # investing_metadata = stock_metadata(ROOT_PATH)
    excluded_stocks = excluded_stock()
    tickers = stock_list_100_highestrank_and_availability()
    tickers = [ticker for ticker in tickers if ticker not in excluded_stocks]
    
    # Total batch per instances
    batch = int(len(tickers) / split)
    
    batch_metadata = tickers[instance_no * batch:(instance_no + 1) * batch]
    
    loops = 0
    for ticker in batch_metadata:
        for output_c in output_cs:
            # Hotstart
            if loops < HOTSTART:
                loops+=1
                continue
            if simtype == 'input_c':
                execute_input_c_v2(ticker, ROOT_PATH, output_c=output_c, model_version=model_version, input_c_limit=input_c_limit, epochs=epochs, rnnus=rnnus, recurrents=recurrents, train=train)
            elif simtype == 'opt':
                execute_configuration_optimization_v2(ticker, ROOT_PATH, output_c=output_c, model_version=model_version, epochs=epochs, rnnus=rnnus, recurrents=recurrents, train=train)
            loops+=1
            
            # Save progress to cache
            with open(progress_cache_file, 'w') as f:
                f.write(str(loops))  
                
def execute_retrain_model_v2(ROOT_PATH='./', output_cs=('close_EMA3_G_s1','close_EMA10_G_s1','close_EMA3_G_s2','close_EMA10_G_s2'), model_version='2', iteration_version ='1', backtest=True):
    '''
    source_db = 'idx_data_v1.3.db'
    For backtest:
        backtest = True
        train = 0.9
    For production:
        backtest = False
        train = 0.99
        
    Revision in v2
    - Create new directory path for easier
        model version grouping
    '''
    if backtest:
        train = 0.9
        STATISTICS_FILE = f'{ROOT_PATH}statistics/v{model_version}/backtest/{iteration_version}.csv'
    else:
        train = 0.99
        STATISTICS_FILE = f'{ROOT_PATH}statistics/v{model_version}/production/{iteration_version}.csv'

    excluded_stocks = excluded_stock()
    tickers = stock_list_100_highestrank_and_availability()
    tickers = [ticker for ticker in tickers if ticker not in excluded_stocks]
    
    for ticker in tickers:
        DB_PATH = f'{ROOT_PATH}db/v{model_version}/idx_{ticker}.db'
        # Load data
        df = load_ml_database(DB_PATH)
        train_df, test_df, train_mean, train_std = split_traintest(df, split=train)

        loops = 0
        for output_c in output_cs:
            tick = datetime.datetime.now()

            # Read optimal input_c configuration
            input_c = input_c_from_sa_v2(ROOT_PATH, model_version, ticker, output_c)

            # Read optimal model configuration
            optimization_statistics_path = f'{ROOT_PATH}statistics/v{model_version}/sa_{ticker}_{output_c}.csv'
            architecture, recurrent, rnnu, epoch = read_optimal_model_config(optimization_statistics_path, indicator='test_rmse')

            # Make dataset
            train_dataset = TimeseriesGenerator(train_df[input_c], train_df[output_c], length=recurrent, shuffle=True)
            test_dataset = TimeseriesGenerator(test_df[input_c], test_df[output_c], length=recurrent)

            # RNN Model
            if architecture == 'SimpleRNN':
                model = simpleRNN_1layer(rnnu, return_sequences=False)
            elif architecture == 'GRU':
                model = GRU_1layer(rnnu, return_sequences=False)
            elif architecture == 'LSTM':
                model = LSTM_1layer(rnnu, return_sequences=False)

            # Feed and train the model
            model.fit(train_dataset, epochs=epoch, verbose=0)

            # Save model weights
            weights_save_path = f'{ROOT_PATH}models/v{model_version}/{iteration_version}/{ticker}_{output_c}/'
            model.save_weights(weights_save_path)

            # Delete current model and re-load weights to make sure that weight is recoverable
            del model
            # RNN Model
            if architecture == 'SimpleRNN':
                model = simpleRNN_1layer(rnnu, return_sequences=False)
            elif architecture == 'GRU':
                model = GRU_1layer(rnnu, return_sequences=False)
            elif architecture == 'LSTM':
                model = LSTM_1layer(rnnu, return_sequences=False)
            model.load_weights(weights_save_path)

            # Evaluate model
            train_eval = model.evaluate(train_dataset, verbose=0)
            test_eval = model.evaluate(test_dataset, verbose=0)

            input_c_joined = '|'.join(input_c)
            train_mean_joined = '|'.join([str(x) for x in list(train_mean[input_c])])
            train_std_joined = '|'.join([str(x) for x in list(train_std[input_c])])
            statistics = f'''{ticker},{input_c_joined},{offset},{output_c},{architecture},{recurrent},{rnnu},{epoch},{train_eval[0]},{train_eval[1]},{test_eval[0]},{test_eval[1]},{train_mean_joined},{train_std_joined},{train_mean[output_c]},{train_std[output_c]}\n'''
            with open(STATISTICS_FILE, 'a') as f:
                f.write(statistics)

            tock = datetime.datetime.now()

            print(f'''{loops} {ticker} {len(input_c)} {offset} {output_c} {architecture} R[{recurrent}] RU[{rnnu}] E[{epoch}] trainRMSE[{round(train_eval[1], 5)}] testRMSE[{round(test_eval[1], 5)}] time: {tock-tick}''')
            loops+=1

# Model development

In [5]:
# Test db read
dbpath = './db/v2/idx_ACES.db'
dbconn = sqlite3.connect(dbpath)
df = pd.read_sql('select * from `1`', dbconn)
df

,time,low_apo_EMA3_EMA30_offset,low_apo_EMA3_EMA200_offset,change_ema_EMA3,change_ema_EMA10,change_ema_EMA30,change_ema_EMA200,change_ema_EMA3_G,change_ema_EMA10_G,change_ema_EMA30_G,...,Volume_rank_macd_EMA10,Volume_rank_macd_EMA30,Volume_rank_macd_EMA200,Volume_rank_macd_EMA3_G,Volume_rank_macd_EMA10_G,Volume_rank_macd_EMA30_G,Volume_rank_macd_EMA200_G,Volume_rank_macd_EMA3_EMA10_offset,Volume_rank_macd_EMA3_EMA30_offset,Volume_rank_macd_EMA3_EMA200_offset
0,1194307200000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1194393600000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1194480000000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1194566400000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1194825600000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3356,1635120000000000000,-33.824412,-9.776075,0.001969,0.002336,0.000993,-0.000119,-0.000659,-0.000228,2.184484e-05,...,0.014848,0.010646,-0.000474,-0.007524,-0.003754,-0.000875,-0.000016,-0.630962,-0.485329,-12.552256
3357,1635206400000000000,22.758182,-11.543489,0.001483,0.002093,0.000993,-0.000108,-0.000486,-0.000244,2.231413e-07,...,0.009856,0.009146,-0.000595,-0.009043,-0.004992,-0.001500,-0.000121,-1.361602,-1.389664,4.989397
3358,1635292800000000000,8.582718,-13.189471,0.001322,0.001924,0.001004,-0.000095,-0.000161,-0.000169,1.087302e-05,...,0.005195,0.007538,-0.000746,-0.006108,-0.004661,-0.001608,-0.000151,-2.861849,-2.283101,11.962839
3359,1635379200000000000,5.161648,-14.946725,0.001205,0.001771,0.001010,-0.000083,-0.000118,-0.000152,5.335220e-06,...,0.001981,0.006246,-0.000863,-0.001404,-0.003214,-0.001292,-0.000117,-6.590879,-2.773204,11.836199
